# Hylogic

Propositional, predicate, and first-order logic evaluator for [Hy](https://github.com/hylang/hy)

## Requirements and installation
<hr/>
### Jupyter and Hy

A little bit work is required to get everything running on your local computer. First you need Jupyter Notebook and Calysto Hy kernel to interact with this document. Easy way to get Jupyter Notebook running is to use Anaconda package from Continuum: https://www.continuum.io/downloads. It will install Python language interpreter to your computer, which is also required.

[Hy](http://docs.hylang.org/en/latest/index.html) language, which by the way is a cool Lisp syntax and feature set upon Python, you can get from: https://github.com/hylang/hy. Install it and then follow Calysto Hy kernel installation instructions from their GitHub project page: https://github.com/Calysto/calysto_hy.

### HyLogic module

Finally you need to retrieve HyLogic module from GitHub: https://github.com/markomanninen/hylogic

After all installations you should be ready to print environment information running the following Hy code:

In [73]:
(import hy sys)
(print "Hy version: " hy.__version__)
(print "Python" sys.version)

Hy version:  0.12.1+24.g45b7a4a
Python 3.5.2 |Anaconda custom (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


### Import main macros

In [74]:
; require macros and import functions and variables
(require (hylogic.macros (*)))
(import [hylogic.macros [*]])
; NL for newlines
(setv NL "\r\n")

## Propositional logic
<hr/>

### Symbols

Propositional constants:

- ⊤ (True / 1)
- ⊥ (False / 0)

### Connectives

Let us list all connectives to operate with the propositional and the first-order logic:

In [75]:
(for [[f data] connectives]
  (print (last data) "\t" (first data) "    \t" (second data)))

¬ 	 not     	 Negation
∧ 	 and     	 Conjunction
↑ 	 nand     	 Nonconjunction
∨ 	 or     	 Disjunction
↓ 	 nor     	 Nondisjunction
↮ 	 xor     	 Exclusive or
↔ 	 xnor     	 Nonexclusive or
≡ 	 eqv     	 Equivalence
≢ 	 neqv     	 Nonequivalence
← 	 cimp     	 Converse implication
↛ 	 cnimp     	 Converse nonimplication
↚ 	 mimp     	 Material implication
→ 	 mnimp     	 Material nonimplication


### Basic axioms and theorems

- Identity $P$ = $P$
- Negation ⊤ = ¬⊥ and ¬⊤ = ⊥

## Propositions

Propositional variables are created with `defproposition` and `defproposition*` macros. Latter macro also creates a negation variable to reduce some repetitive work. In `HyLogic` a proposition consists of:

1) a propositional variable that is usually denoted by a capital letter like $P$, $Q$ and $R$

2) a truth value that is either $True$ or $False$. Truth value can also be defined by using the number $1$ for $True$ and the number $0$ for $False$ or by using constant symbols $⊤$ and $⊥$ respectively

3) a sentence (statement), or a literal representation of the proposition such as the phrase "Today is Tuesday"

#### Example 1

Let us define two propositions and their negations by using `defproposition*` macro. Format is the following: `(macro symbol truth-value statement)`.

In [76]:
(defproposition* P False "Today is Tuesday")
(defproposition* Q True "John will go to work")

And then print out the propositions:

In [77]:
(print P ¬P NL)
(print Q ¬Q)

P<Today is Tuesday>=False ¬P<Today is Tuesday>=True 

Q<John will go to work>=True ¬Q<John will go to work>=False


From the output we find each proposition and its negation represented in a string format that distinquishes all three aspects of the proposition namely the symbol, the literal representation and the truth value of the proposition.

### Explanation

Maybe a small explanation here is in place because understanding the basic components of the propositional logic requires the understanding of the common convention on how propositional logic works and how it is represented in a written or a spoken format.

When we define a proposition $P$ (that is just a freely selected symbol) to mean "Today is Tuesday" and to be False, the following happens. We define that the abbreviation  $P$ denotes the sentence "Today is Tuesday" in human language. We also define that the statement i.e. its truth value is $False$. Thus we could understand the proposition $P$ to say something like "Today is not Tuesday". But there is a possible pitfall in this expression. Strictly speaking the proposition $P$ is just saying that the statement "Today is Tuesday" is $False$, nothing more. We are not trying to determine if it is really Tuesday today, or if the statement is really true. In some sense logic is not meant to find out truth from the world, but to help to follow logical steps to determine the consequence of the predefined statements. So, we define $P$ to be $False$ and deduce the rest of the argumentation according to that definition.

However, the negation of the proposition $P$, which is automaticly generated by the `defproposition*` macro, is $¬P$ (not $P$). By literal representation it could be written: "It is not the case that Today is Tuesday". `HyLogic` library doesn't try to formulate literal representations of the negated statements. Library just formulates negations by prepending $¬$ symbol to the propositional variable and switching the truth value to its opposite.

The truth value of the negation of $P$ in question is of course $True$ because the original $P$ was defined to be $False$ and because we defined negation to work such way in the basic axioms and theorems.

## Argumentation
<hr/>

Introducing `defargument`, `defpremise`, and `defconclusion` macros.

In propositional logic, argument is a set of premises following each other where the final premise is called the conclusion. This is also called deductive reasoning where the more spesific conclusion is reasoned from the more general premises or assumptions. In `HyLogic` argument is created by `defargument` macro which returns an object that can be assigned to a variable for further interaction. `Defargument` takes a serie of premises defined by `defpremise` macro plus the final conclusion defined by `defconclusion` macro. Each premise is a set of propositions and axioms constructed by [inference rules](https://en.wikipedia.org/wiki/Rule_of_inference).

#### Example 2

The next example is meant to demonstrate argumentation process in `HyLogic`. It is the famous [modus ponens](https://en.wikipedia.org/wiki/Modus_ponens) implication elimination rule. But there is a small twist that should stress the need of accuracy on small details in logical reasoning.

First we will define the implication premise "if P is True, then Q is True". Then we will define the second premise "P" and finally the conclusion "Q".

In [78]:
(setv a 
  (defargument 
    ; If the proposition "Today is Tuesday" (P) is True
    ; then the proposition "John will go to work" (Q) must be True as well.
    (defpremise P → Q)
    ; But we stated earlier on example 1 that the proposition "Today is Tuesday" (P) is False.
    ; How should we deal with it now?
    (defpremise P)
    ; Well therefore, both <John will go to work>=True 
    ; and <John will go to work>=False should be concluded as a valid argument.
    (defconclusion Q)))
;(print a)

It means that if Today is Tuesday is False OR "John will go to work" is True, then premise is True

Thus if "Today is not Tuesday" then "John will go to work" or "John will not go to work"

Moreover, because P is False, it tells nothing about Q so we can accept both True and False statements of Q

In [79]:
(defproposition* P False "Today is Tuesday")
(defproposition* Q True "John will go to work")

(print
  (deffix P) (deffix Q) NL
  "If P, then Q =" (deffix (P → Q)))

P<Today is Tuesday>=False Q<John will go to work>=True 
 If P, then Q = True


In [80]:
(defn gs [s] (if (= s True) s.symbol (+ "¬" s.symbol)))

(print (gs P) " → " (gs Q) "\t\t\t" (deffix P → Q))
(print (gs P) "\t\t\t\t" (deffix P = True))
;(print "(" (gs P) " → " (gs Q) ") ∧ " (gs P) "\t\t" (deffix (P → Q) ∧ P))
(print (gs Q) "\t\t\t\t" (deffix Q = True))
(print "(((" (gs P) " → " (gs Q) ") ∧ " (gs P) ") → " (gs Q) ")\t" (deffix (((P → Q) ∧ P) → Q)))

¬P  →  Q 			 True
¬P 				 False
Q 				 True
((( ¬P  →  Q ) ∧  ¬P ) →  Q )	 True


#### Example 3

Slightly more complicated argument is shown next.

In [81]:
(defproposition P True "It is raining")
(defproposition Q True "It is cold outside")
(defproposition R False "I'm indoors")

(print P NL Q NL R)

P<It is raining>=True 
 Q<It is cold outside>=True 
 R<I'm indoors>=False


In [82]:
; set up argument inference rules
(setv a 
  (defargument 
    ; If "it is raining and it is cold outside" then "I'm indoors"
    (defpremise (P ∧ Q) → R)
    ; It is raining and it is cold outside
    (defpremise (P ∧ Q))
    ; Therefore, I'm indoors
    (defconclusion R)))

In [83]:
(print a)


  (P ∧ Q) → R
  (P ∧ Q)
--------------
∴ R



In [84]:
(print
  (deffix P) (deffix Q) (deffix R) NL
  "If P and Q, then R =" (deffix (P ∧ Q) → R))

P<It is raining>=True Q<It is cold outside>=True R<I'm indoors>=False 
 If P and Q, then R = False


## First order logic
<hr/>
### Quantifiers, predicates, variables, sets

#### Universal quantifier (∀)

In [85]:
;(∀x∈ℕ>0∧<10)(x>0)
; (all (map (fn (x) (x > 0)) (range 1 10)))
(∀ (x) (x > 0) (range 1 10)) ; all items [1 ... 9] are greater than 0?

True

In [86]:
;all(map(lambda x: x > 0, range(1 10)))
(all (map (fn (x) (> x 0)) (range 1 10)))

True

In [87]:
(∀ (x y) ((x > 0) ∧ (y < 0)) (range 1 10) (range -10 -1))

True

In [88]:
(∀ (x) (∀ (y) ((x > 0) ∧ (y < 0)) (range 1 10)) (range -10 -1))

False

In [89]:
(∀ (x) ((x > 0) ∧ (∀ (y) (y < 0) (range 1 10))) (range -10 -1))
;(macroexpand `(∀ (x) ((x > 0) ∧ (∀ (y) (y < 0) (range 1 10))) (range -10 -1)))

False

In [90]:
; Every whole number is divisible by 1 and itself.
;(∀x)(Div(x,x)∧(Div(1,x))

;(defoperator mod [x y] (% x y))
(defoperator mod0? [x y] (zero? (% x y)))
(defoperator Div [x y] (mod0? x y))

(setv domain-set [1 2 3])

(∀ (x) ((Div x 1) ∧ (Div x x)) domain-set)

True

In [91]:
(setv DX [1 1]
      DY [-1 -2])
; all[any[1-1=0 1-2=-1] any[1-1=0 1-2=-1]]
(∀ (x) (∃ (y) (x + y = 0) DY) DX)

True

In [92]:
; all[1-1=0 1-2=-1 1-1=0 1-2=-1]
(∀ (x y) (x + y = 0) DX DY)

False

#### Existential  quantifier (∃)

In [93]:
(∃ (x) (x < 1) (range 0 10)) ; is at least one item of [0 ... 9] smaller than 1?

True

In [94]:
; (∃x)((¬Div(x,x))∨(¬Div(1,x))
(∃ (x) ( (¬ (x mod0? 1) ) ∨ (¬ (x mod0? x) ) ) domain-set)

False

In [95]:
; any[1-1=0 1-2=-1 1-1=0 1-2=-1]
(∃ (x y) ((x > 0) ∧ (y < 0)) DX DY)

True

In [96]:
; any[all[1-1=0 1-2=-1] all[1-1=0 1-2=-1]]
(∃ (x) (∀ (y) (x + y = 0) DY) DX)

False

## Truth tables
<hr/>

In [97]:
(truth-tables-html 2 cimp?)

In [98]:
(truth-tables-html 2 eqv?)

In [99]:
(truth-tables-html 2 xnor?)

## Venn diagrams
<hr/>

In [100]:
;(venn-diagram)

In [101]:
(defn odd? [x &rest y]
  (= 1 (% (+ x (sum y)) 2)))

(deffix (odd? 1 1 1))

True

### The [MIT](https://choosealicense.com/licenses/mit/) License

Copyright © 2017 Marko Manninen